In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import random
from kafka import KafkaConsumer, KafkaProducer
from json import loads, dumps
from collections import Counter

import sys
sys.path.append('./CountMinSketch/')

from countminsketch import CountMinSketch
from hashfactory import hash_function

In [2]:
seed=0
np.random.seed(seed)
random.seed(seed)

In [3]:
file = pd.read_csv('november_2021_COVID-19_Twitter_Streaming_Dataset.csv')

In [4]:
def find_tweet_timestamp_post_snowflake(tid):
    offset = 1288834974657
    tstamp = (tid >> 22) + offset
    return tstamp

In [5]:
# hash 함수 개수
DEPTH = 5
# WIDTH = 2**22
# 데이터 수
WIDTH = 7
HASH_FUNCTIONS = [hash_function(i) for i in range(DEPTH)]

In [6]:
len(HASH_FUNCTIONS)

5

In [7]:
hash_map = []
for i in range(len(HASH_FUNCTIONS)):
    list_=[]
    for j in range((WIDTH)):
        list_.append(HASH_FUNCTIONS[i](j)%WIDTH)
        print(HASH_FUNCTIONS[i](j)%WIDTH, end=' ')
    hash_map.append(list_)
    print()

5 6 1 4 5 2 6 
4 5 4 0 2 0 1 
6 4 5 0 1 0 3 
0 1 5 5 4 3 5 
1 4 2 2 5 2 2 


In [8]:
hash_map =np.array(hash_map)
hash_map.shape

(5, 7)

In [9]:
WEEKDAY = {'Mon':0,
           'Tue':1,
           'Wed':2,
           'Thu':3,
           'Fri':4,
           'Sat':5,
           'Sun':6
          }

In [15]:
batch = Counter()
sketch = CountMinSketch(DEPTH, WIDTH, HASH_FUNCTIONS)
for idx, tid in enumerate(file['_id']):
    # print(tid)
    timestamp = find_tweet_timestamp_post_snowflake(tid)
    current_date = datetime.datetime.fromtimestamp(timestamp/1000)
    
    hour = current_date.hour
    weekday = WEEKDAY[current_date.ctime().split()[0]]
    #print(current_date.hour, current_date.ctime().split()[0])
    
    batch[weekday]+=1
    
    for key, count in batch.items():
        # print(key, count)
        sketch.add(key, count)
    batch.clear()
    
    # if idx==100:
    #     break

KeyboardInterrupt: 

In [16]:
hash_map

array([[5, 6, 1, 4, 5, 2, 6],
       [4, 5, 4, 0, 2, 0, 1],
       [6, 4, 5, 0, 1, 0, 3],
       [0, 1, 5, 5, 4, 3, 5],
       [1, 4, 2, 2, 5, 2, 2]])

In [17]:
sketch.get_matrix()

array([[      0,       0,       0,       0,       0, 2021286, 2678052],
       [      0,       0,       0,       0, 2021286, 2678052,       0],
       [      0,       0,       0,       0, 2678051,       0, 2021286],
       [2021286, 2678051,       0,       0,       0,       0,       0],
       [      0, 2021286,       0,       0, 2678051,       0,       0]],
      dtype=int32)

In [18]:
# Monday to Sunday
for index in range(WIDTH):
    print(min([sketch.get_matrix()[i][j] for i,j in enumerate([j for j in hash_map[:, index]])]))

2021286
2678051
0
0
0
0
0


In [19]:
tid

1455495752537444356